## Import Necessary Modules

In [5]:
import os
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [6]:
# Load the saved CSV file into a new DataFrame
df = pd.read_csv('transformed_sales_data.csv')

### Convert the date column back into the right dtype

In [7]:
print(df.info())   # When converted into a CSV, the date column doesn't recognize it should be a datetime object

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')  # reconverting it back to a datetime is required
print('______________________________________________________')

print(df.info())  # Now the date column is recognized as a datetime object, enabling more advanced analysis.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      997 non-null    object 
 1   category  997 non-null    object 
 2   sales     997 non-null    float64
 3   region    997 non-null    object 
dtypes: float64(1), object(3)
memory usage: 31.3+ KB
None
______________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      997 non-null    datetime64[ns]
 1   category  997 non-null    object        
 2   sales     997 non-null    float64       
 3   region    997 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 31.3+ KB
None


### Connecting and loading data into the database

- Load environment variables
    - `load_dotenv()` loads variables from the `.env` file to ensure there is no hard coded credentials
    - `os.getenv()` retrieves the value of an environment variable by its name
- Create a connection string to the RDS database
    - `f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}'` creates a connection string that can be used to connect to the RDS database
    - `create_engine()` creates a database engine using a string that specifies the database type and connection details
- Testing the connection
    - `engine.connect()` checks if the connection is successful and prints a confirmation message
- Create database (if not already created)
    - `with connection.connection.cursor() as cursor:` creates a cursor object that can execute SQL commands
    - Create a new data base called 'sales' if it does not already exist and the print a confirmation message
- Update the connection string to the new `sales` database
- Load transformed  dataframe into the new `sales` database into a table called `sales_data` using the `.to_sql()` function
- Print final confirmation message!


In [10]:
import os
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()


# Database connection details
db_host =  os.getenv('db_host') 
db_name = 'mysql'
db_user = os.getenv('db_username')  
db_password = os.getenv('db_password') 

# Create a connection to RDS

connection_string = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}'
engine = create_engine(connection_string)

with engine.connect() as connection:
    print("Connection successful!")
    
    with connection.connection.cursor() as cursor:
        # SQL to create a database
        query = "CREATE DATABASE IF NOT EXISTS sales;"
        cursor.execute(query)
        print("Database 'sales' created successfully!")
        
# Update connection string to point to the new database
sales_connection_string = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/sales'
sales_engine = create_engine(sales_connection_string)

# Load transformed data into the new database
df.to_sql('sales_data', con=sales_engine, if_exists='replace', index=False)
print("Data loaded into the 'sales_data' table successfully!")


Connection successful!
Database 'sales' created successfully!
Data loaded into the 'sales_data' table successfully!


### Query to test sales_data table

In [29]:
import pandas as pd
from sqlalchemy import create_engine

# Load variables from .env file
load_dotenv()

# Connection string to the sales database
db_host = os.getenv('db_host')
db_name = 'sales'
db_user = os.getenv('db_username')
db_password = os.getenv('db_password')

connection_string = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}'
engine = create_engine(connection_string)

# Query the sales_data table
try:
    with engine.connect() as connection:
        df = pd.read_sql("SELECT * FROM sales_data", con=connection)
        print("Data from sales_data table:")
        print(df)
except Exception as e:
    print("An error occurred:", e)



Data from sales_data table:
          date     category   sales region
0   2023-04-09     Clothing  973.80  South
1   2023-05-31  Electronics  500.00  North
2   2023-01-08         Toys  133.06  North
3   2023-04-27         Home  246.14  South
4   2023-08-30         Toys  253.09  South
..         ...          ...     ...    ...
992 2023-03-25         Home  798.74   West
993 2023-06-03         Home  708.32   East
994 2023-06-22  Electronics  265.83  South
995 2023-10-28  Electronics   98.78   West
996 2023-11-08         Home  109.26  North

[997 rows x 4 columns]
